In [7]:
import sqlite3
import sql_queries as sq
import pandas as pd

db_conn = sqlite3.connect('./../../_data_raw/fa_database.db')

In [16]:
# create tables in create_table_list
for table in sq.create_table_list:
    cursor = db_conn.cursor()
    
    try:
        cursor.execute(table)
        db_conn.commit()
        cursor.close()
    except Exception as e:
        print(e)
        db_conn.rollback()
        cursor.close()

In [2]:
# drop tables in create_table_list
#for table in sq.drop_table_list:
#    cursor = db_conn.cursor()
#    
#    try:
#        cursor.execute(table)
#        db_conn.commit()
#        cursor.close()
#    except Exception as e:
#        print(e)
#        db_conn.rollback()
#        cursor.close()

# DROP TABLE MANUALLY
# table_to_drop = 'ov_revenue'
# cursor = db_conn.cursor()
# cursor.execute('drop table if exists %s' %table_to_drop)
# cursor.commit()

In [10]:
## test extract and load data from bigquery to local db
from  class_ETL import ETL

worker = ETL(
            credentials_file_path= './../../_config/cred.json',
             local_database= './../../_data_raw/fa_database.db'
             )


In [2]:
df_bigquery_data = worker.extract_bigquery_data(start_date= '2024-01-01')
df_bigquery_data

,complete_date,province,service_type,customer_type,partner,project,gsv_excluded_vat,stoppoint,aha_revenue
0,2024-01-01,sgn,truck,sme,NULL,truck,1.304903e+08,551.0,1.941801e+07
1,2024-01-01,dad,bike,sme,NULL,expansion,3.782801e+07,1497.0,7.697593e+06
2,2024-01-01,vcs,bike,sme,NULL,expansion,9.565815e+06,470.0,2.002037e+06
3,2024-01-01,han,bike,ka_bd_ops,MemoryZone,core,4.628889e+05,13.0,9.185185e+04
4,2024-01-01,dni,bike,sme,NULL,expansion,1.732915e+07,533.0,3.742407e+06
...,...,...,...,...,...,...,...,...,...
47260,2024-04-13,bmv,bike,ka_bd_ops,Con cưng,expansion,4.859259e+05,20.0,9.318519e+04
47261,2024-04-13,dni,bike,ka_bd_ops,Con cưng,expansion,1.819167e+06,84.0,3.618333e+05
47262,2024-04-13,sgn,truck,ka_bd_ops,Gym Store,truck,2.718519e+05,1.0,4.055556e+04
47263,2024-04-13,vii,bike,ka_bd_ops,Con cưng,expansion,7.287037e+04,3.0,1.457407e+04


In [11]:
df_fct_fa_pnl_mini = worker.extract_local_data(table_name='fct_fa_pnl_mini')
df_fct_fa_pnl_mini

[('2021-01-01',
  'han',
  'bike',
  'KA (BD + OPS)',
  'HEETNUSO HN',
  'core',
  1702360.0,
  63.0,
  311759.99999999994,
  '2024-04-14 08:09:57'),
 ('2021-01-01',
  'sgn',
  'bike',
  'KA (BD + OPS)',
  'Đà Lạt Hasfarm',
  'core',
  328500.0,
  11.0,
  58499.999999999985,
  '2024-04-14 08:09:57'),
 ('2021-01-01',
  'han',
  'bike',
  'KA (BD + OPS)',
  'Mẹ Suri Shop',
  'core',
  227360.0,
  6.0,
  41759.99999999999,
  '2024-04-14 08:09:57'),
 ('2021-01-01',
  'han',
  'bike',
  'KA (BD + OPS)',
  'Luôn Tươi Sạch',
  'core',
  401800.0,
  15.0,
  73799.99999999999,
  '2024-04-14 08:09:57'),
 ('2021-01-01',
  'han',
  'bike',
  'KA (BD + OPS)',
  'HEETNUSO HN',
  'core',
  1702360.0,
  63.0,
  311759.99999999994,
  '2024-04-14 08:14:49'),
 ('2021-01-01',
  'sgn',
  'bike',
  'KA (BD + OPS)',
  'Đà Lạt Hasfarm',
  'core',
  328500.0,
  11.0,
  58499.999999999985,
  '2024-04-14 08:14:49'),
 ('2021-01-01',
  'han',
  'bike',
  'KA (BD + OPS)',
  'Mẹ Suri Shop',
  'core',
  227360.0,
  6

In [4]:
# test load data to local db with specific start_date to insert
worker.load_data(data_ingest= df_bigquery_data.to_records(index=False).tolist(), table_name= 'fct_fa_pnl_mini' )

In [3]:
# test load data automatically from lastest complete_date in fct_fa_pnl_mini to current_date - 1 day
worker.auto_ingest_data(table_name='fct_fa_pnl_mini')

In [10]:
import requests

res = requests.get(f'https://api.vietqr.io/v2/business/0107370860',timeout=1000)
res
res.json()

{'code': '00',
 'desc': 'Success - Thành công',
 'data': {'id': '0107370860',
  'name': 'CÔNG TY TNHH ĐẦU TƯ THƯƠNG MẠI HOÀNG MINH VIỆT NAM',
  'internationalName': 'HOANG MINH VIET NAM INVESTMENT TRADING COMPANY LIMITED',
  'shortName': None,
  'address': 'Số 887 Giải Phóng, Phường Giáp Bát, Quận Hoàng Mai, Thành phố Hà Nội, Việt Nam'}}